In [49]:
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [70]:
lats = np.random.uniform(low=-90, high=90, size=2000)
lngs = np.random.uniform(low=-180, high=180, size=2000)
lat_lng = zip(lats, lngs)
lat_lng

In [71]:
coordinates = list(lat_lng)

In [72]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [73]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

748

In [76]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_des = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Current Description": city_weather_des,
                         "Country": city_country})
        
        # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | akhmim
Processing Record 2 of Set 1 | karwar
Processing Record 3 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 4 of Set 1 | hithadhoo
Processing Record 5 of Set 1 | hofn
Processing Record 6 of Set 1 | bathsheba
Processing Record 7 of Set 1 | pevek
Processing Record 8 of Set 1 | cidreira
Processing Record 9 of Set 1 | arraial do cabo
Processing Record 10 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 11 of Set 1 | yambio
Processing Record 12 of Set 1 | hasaki
Processing Record 13 of Set 1 | tiznit
Processing Record 14 of Set 1 | vaini
Processing Record 15 of Set 1 | isfana
Processing Record 16 of Set 1 | laguna
Processing Record 17 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 18 of Set 1 | kyabe
Processing Record 19 of Set 1 | isangel
Processing Record 20 of Set 1 | phnum penh
City not found. Skipping...
Processing Record 21 of Set

In [80]:
city_data_df = pd.DataFrame(city_data)

In [89]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", 
                    "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Akhmim,EG,26.5622,31.7450,86.20,37,0,14.32,clear sky
1,Karwar,IN,14.8000,74.1333,77.59,87,98,2.21,overcast clouds
2,Hithadhoo,MV,-0.6000,73.0833,82.56,68,100,17.74,overcast clouds
3,Hofn,IS,64.2539,-15.2082,42.82,56,90,11.50,overcast clouds
4,Bathsheba,BB,13.2167,-59.5167,88.41,66,75,12.66,broken clouds
5,Pevek,RU,69.7008,170.3133,27.95,82,100,9.60,overcast clouds
6,Cidreira,BR,-30.1811,-50.2056,66.13,75,31,10.22,scattered clouds
7,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,56,0,32.21,clear sky
8,Yambio,SS,4.5721,28.3955,67.24,96,95,3.94,overcast clouds
9,Hasaki,JP,35.7333,140.8333,70.29,74,100,16.93,overcast clouds


In [90]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")